In [3]:
import boto3
boto3.__version__

'1.21.32'

In [22]:
##creating athena client 
client = boto3.client('athena')
print(client.list_databases)

<bound method ClientCreator._create_api_method.<locals>._api_call of <botocore.client.Athena object at 0x000001DDABDA0790>>


In [95]:
## creating response from athena 
import time
response=client.start_query_execution(
    QueryString='SELECT gender,count(*) FROM "data-engineer"."claims" Group by gender ; ',
    QueryExecutionContext= {
        'Database': 'data-engineer'
    },
    ResultConfiguration={
        'OutputLocation':'s3://techkraft-dataengineer/athena-results/'
    }
)
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('Query successfully executed')
queryExecutionId=response['QueryExecutionId']

time.sleep(20)

result= client.get_query_results(QueryExecutionId= queryExecutionId)


Query successfully executed


In [102]:
## extracting headers 
key = []
value = []
data = {}
count = 0
for   row in result['ResultSet']['Rows']:
    dataframe=row['Data']
    # print(len(dataframe))
    if count==0:
        for i in range(len(dataframe)):
            header=dataframe[i]['VarCharValue']
            key.append(header)
    count=count+1

In [104]:
## extracting body of table and converting into dict 
for k in key:
    data[k] = []
for row in result['ResultSet']['Rows'][1:]:
    for i,k in enumerate(key):
        print(i,k)
        data[k].append(row['Data'][i]['VarCharValue'])


0 gender
1 _col1
0 gender
1 _col1


In [101]:
import pandas as pd
df=pd.DataFrame(data)
df.to_csv('output.csv',index=False)

In [ ]:
## creating s3 client 
client1 = boto3.client('s3')
response=client1.list_buckets()
print(response)
# response = client1.list_buckets(client1)

In [ ]:
## to upload file 
client1.upload_file('output.csv','trainee-2022','rajiv_task/count_m_f.csv')

In [ ]:
## to delete all file 
resource = boto3.resource('s3')

your_bucket = resource.Bucket('trainee-2022')
files = list(your_bucket.objects.all())
print((files))
for file in files:

    if 'rajiv_task/' in file.key:
        print(file.key)
        
        client1.delete_object(Bucket='trainee-2022',Key=file.key)

In [25]:
## download file from athena 
output2=response['QueryExecutionId']
client1.download_file('techkraft-dataengineer',f'athena-results/{output2}.csv','output2.csv')